Combine CSVs and remove duplicates

In [ ]:
# SECTION: Install missing dependencies
print("Checking for and installing missing dependencies...")

import sys
!{sys.executable} -m pip install pandas python-dotenv

print("Done installing dependencies. Please re-run the notebook from the top if needed.")


In [ ]:
# SECTION: Import libraries
print("Importing libraries...")
import os
from dotenv import load_dotenv
import pandas as pd
from glob import glob
import re

# SECTION: Load environment variables
print("Loading environment variables from .env...")
load_dotenv()
CSV_FOLDER = os.getenv("CSV_FOLDER")
SUMMARY_FOLDER = os.getenv("SUMMARY_FOLDER")

if not CSV_FOLDER or not SUMMARY_FOLDER:
    raise ValueError("Could not find CSV_FOLDER or SUMMARY_FOLDER in .env.")

print(f"CSV_FOLDER: {CSV_FOLDER}")
print(f"SUMMARY_FOLDER: {SUMMARY_FOLDER}")

# SECTION: Define canonical column mapping (all known variants)
column_map = {
    'first_author':   ['first_author', 'first author', 'author'],
    'all_authors':    ['all_authors', 'all authors', 'authors'],
    'title':          ['title', 'paper title'],
    'abstract':       ['abstract', 'summary'],
    'year':           ['year', 'publication year', 'pub_year', 'date'],
    'doi':            ['doi', 'arxiv id', 'arxiv_id'],
}

required_final_cols = [
    'first_author', 'all_authors', 'title', 'abstract',
    'year', 'doi', 'source', 'record type', 'dedup_key'
]

# flatten aliases for fast lookup
flat_map = {}
for canon, aliases in column_map.items():
    for a in aliases:
        flat_map[a.lower()] = canon

# SECTION: Identify latest-version CSVs only
print("\nFinding all CSV files in the folder...")
csv_files = sorted(glob(os.path.join(CSV_FOLDER, "*.csv")))
if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {CSV_FOLDER}")

print("All CSV files found:")
for fname in csv_files:
    print(" ", os.path.basename(fname))

def extract_base_and_version(filename):
    m = re.match(r'(.+?)_v(\d+)\.csv$', filename)
    if m:
        base = m.group(1)
        ver = int(m.group(2))
        return base, ver
    else:
        base = filename[:-4] if filename.lower().endswith('.csv') else filename
        return base, 0

versioned_files = {}
for fpath in csv_files:
    fname = os.path.basename(fpath)
    base, ver = extract_base_and_version(fname)
    if base not in versioned_files or ver > versioned_files[base][1]:
        versioned_files[base] = (fpath, ver)
selected_files = [tpl[0] for tpl in versioned_files.values()]

print("\nFiles selected for combining (latest version of each base name):")
for fname in selected_files:
    print(" ", os.path.basename(fname))

# User confirmation
proceed = input("\nProceed with these files? Type 'y' and Enter to continue, or anything else to stop: ")
if proceed.strip().lower() != 'y':
    print("Aborting as per user request.")
    raise SystemExit()

# SECTION: Read, align, and annotate CSVs
print("\nReading, aligning, and annotating each CSV file...")
file_records = []
for csv_file in selected_files:
    try:
        df_orig = pd.read_csv(csv_file, dtype=str)
        source_name = os.path.basename(csv_file)
        # Map columns
        newcols = {}
        for c in df_orig.columns:
            c_key = c.lower().strip()
            if c_key in flat_map:
                newcols[c] = flat_map[c_key]
        df = df_orig.rename(columns=newcols)

        # Add missing columns as blank
        for canoncol in column_map.keys():
            if canoncol not in df.columns:
                df[canoncol] = ''

        # dedup: use doi if any not-empty, otherwise title
        has_doi = ('doi' in df.columns) and df['doi'].str.strip().any()
        has_title = ('title' in df.columns) and df['title'].str.strip().any()
        if has_doi:
            dedup_key = 'doi'
            df['dedup_id'] = df['doi']
        elif has_title:
            dedup_key = 'title'
            df['dedup_id'] = df['title']
        else:
            raise ValueError(f"File {source_name} missing both usable 'doi' and 'title' columns!")
        df['file_tag'] = source_name
        df['dedup_key'] = dedup_key
        file_records.append(df)
        print(f"  Read {source_name} ({len(df)} rows, dedupe by '{dedup_key}')")
    except Exception as e:
        print(f"  Error reading {csv_file}: {e}")

if not file_records:
    raise Exception("No valid CSVs read. Aborting.")

# SECTION: Combine, aggregate and deduplicate
print("\nCombining records and annotating duplicates...")
combined_df = pd.concat(file_records, ignore_index=True)

# Group: list of source files for each dedup value
dedup_files = (
    combined_df
    .groupby('dedup_id')['file_tag']
    .agg(lambda files: ";".join(sorted(set(files))))
    .reset_index()
    .rename(columns={'file_tag': 'source'})
)

# count appearances
dedup_counts = (
    combined_df
    .groupby('dedup_id')
    .size()
    .reset_index(name='dedup_count')
)

# merge back
combined_annotated = pd.merge(combined_df, dedup_files, on='dedup_id', how='left')
combined_annotated = pd.merge(combined_annotated, dedup_counts, on='dedup_id', how='left')

combined_annotated['record type'] = combined_annotated['dedup_count'].apply(lambda c: "unique" if c == 1 else "duplicate")

# Drop duplicated dedup_id, keeping first
final_df = (
    combined_annotated
    .drop_duplicates(subset=['dedup_id'])
    .reset_index(drop=True)
)

# Arrange columns (ensure all present)
for col in required_final_cols:
    if col not in final_df.columns:
        final_df[col] = ''

final_df = final_df[required_final_cols]

print(f"Final combined CSV has {len(final_df)} unique deduplicated records.")

# SECTION: Save combined CSV
os.makedirs(SUMMARY_FOLDER, exist_ok=True)
output_path = os.path.join(SUMMARY_FOLDER, "combined_csv.csv")
final_df.to_csv(output_path, index=False)
print(f"Combined CSV written to: {output_path}")

# (Optional) Display the first few rows for inspection
final_df.head()
